In [ ]:
!pip install kaggle
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

from google.colab import drive
drive.mount('./gdrive')

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 64 bytes
Mounted at ./gdrive


In [ ]:
!pip install -q pytorch-lightning & pip install -q -U albumentations & pip install -q iterative-stratification
!pip install -q timm & pip install -q einops & pip install -q pytorch-lightning wandb & pip install torch-ema
!git clone https://github.com/mlpc-ucsd/CoaT

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.6/890.6 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.8/225.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.4/291.4 kB 20.7 MB/s eta 0:00:00
Cloning into 'CoaT'...
remote: Enumerating objects: 1339, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 1339 (delta 10), reused 8 (delta 8), pack-reused 1325 (from 1)
Receiving objects: 100% (1339/1339), 7.10 MiB | 32.18 MiB/s, done.
Resolving deltas: 100% (464/464), done.


In [ ]:
!pip install -q pydicom==2.4.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 22.9 MB/s eta 0:00:00


In [ ]:
%%time
!kaggle competitions download -c rsna-2024-lumbar-spine-degenerative-classification
!unzip -q rsna-2024-lumbar-spine-degenerative-classification.zip

100% 28.2G/28.2G [01:48<00:00, 256MB/s]
100% 28.2G/28.2G [01:48<00:00, 279MB/s]
CPU times: user 4.07 s, sys: 643 ms, total: 4.72 s
Wall time: 9min 3s


In [ ]:
!kaggle datasets download -d wadakoki/rsna-newmeta
!unzip -q /content/rsna-newmeta.zip

Dataset URL: https://www.kaggle.com/datasets/wadakoki/rsna-newmeta
License(s): unknown
  0% 0.00/3.76M [00:00<?, ?B/s]
100% 3.76M/3.76M [00:00<00:00, 231MB/s]


In [ ]:
!kaggle datasets download -d wadakoki/extended-train-label-coordinates
!unzip -q /content/extended-train-label-coordinates.zip

Dataset URL: https://www.kaggle.com/datasets/wadakoki/extended-train-label-coordinates
License(s): unknown
  0% 0.00/1.31M [00:00<?, ?B/s]
100% 1.31M/1.31M [00:00<00:00, 167MB/s]


In [ ]:
import wandb
from pytorch_lightning.loggers import WandbLogger
import os
import yaml
import sys
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from glob import glob
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.optim import AdamW, Adam
from torch.optim.lr_scheduler import CosineAnnealingLR, StepLR, ReduceLROnPlateau
import torch.nn as nn
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, TQDMProgressBar
import torchvision.transforms as T
import albumentations as A
import pandas.api.types
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold, StratifiedGroupKFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
#from torchvision.models.maxvit import MaxVit
from timm.models.maxxvit import MaxxVit
import timm
import scipy
import albumentations as A
from torchvision.transforms import v2
from torchvision import models
from einops import repeat
from einops.layers.torch import Rearrange
from tqdm.auto import tqdm
#sys.path.append('/content/CoaT')
#from CoaT.src.models.coat import coat_lite_medium
from joblib import Parallel, delayed
from torch.utils.data import default_collate
from torch_ema import ExponentialMovingAverage
import pydicom as dcm
import transformers

## training

In [ ]:
SEED = 126 # friend's birthday
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True # Fix the network according to random seed
    print('Finish seeding with seed {}'.format(seed))

seed_everything(SEED)
print('Training on device {}'.format(device))

Finish seeding with seed 126
Training on device cuda


In [ ]:
%%writefile config.yaml

data_path: "/content/"
output_dir: "/content/gdrive/MyDrive/RSNA_SPINE/models/"

seed: 1101

train_bs: 2
valid_bs: 2
workers: 1

progress_bar_refresh_rate: 1

pseudo_train: 0

save_topk: 1
fold: 5

task:
    #kind: 'detect'
    kind: 'classify'
    #kind: 'depth'
    #kind: 'depth2'
    #condition: 'nfn'
    condition: 'scs'
    #condition: 'ss'

image_size: 384
early_stop:
    monitor: "val_loss"
    mode: "min"
    patience: 999
    verbose: 1

trainer:
    max_epochs: 7
    min_epochs: 7
    enable_progress_bar: True
    precision: "16-mixed"
    devices: 1

model:
    name: "eff"
    loss_smooth: 0.0
    optimizer_params:
        #lr: 0.001
        lr: 0.00007
        weight_decay: 0.0001
    scheduler:
        #name: "CosineAnnealingLR"
        name: "cosine_with_warmup"
        params:
            CosineAnnealingLR:
                T_max: 20
                eta_min: 1.0e-5
                last_epoch: -1
            cosine_with_warmup:
                num_training_steps: 8
                num_warmup_steps: 1
                num_cycles: 0.5
                last_epoch: -1

Overwriting config.yaml


In [ ]:
with open("config.yaml", "r") as file_obj:
    config = yaml.safe_load(file_obj)

In [ ]:
class ClassDataset(Dataset):
    def __init__(self, df, series, coor, meta, condition, usage='train'):
        #if condition == 'scs':
        #    coor = self.sagt22sagt1(coor, meta)
        self.series = series
        self.coor = coor
        self.meta = meta
        self.df = df
        self.condition = condition
        self.usage = usage
        self.sag_window = 5
        self.ax_window = 5
        if condition == 'scs':
            self.label = [
                'spinal_canal_stenosis_l1_l2', 'spinal_canal_stenosis_l2_l3', 'spinal_canal_stenosis_l3_l4', 'spinal_canal_stenosis_l4_l5', 'spinal_canal_stenosis_l5_s1'
            ]
        elif condition == 'ss':
            self.label = [
                'left_subarticular_stenosis_l1_l2', 'left_subarticular_stenosis_l2_l3', 'left_subarticular_stenosis_l3_l4', 'left_subarticular_stenosis_l4_l5', 'left_subarticular_stenosis_l5_s1',
                'right_subarticular_stenosis_l1_l2', 'right_subarticular_stenosis_l2_l3', 'right_subarticular_stenosis_l3_l4', 'right_subarticular_stenosis_l4_l5', 'right_subarticular_stenosis_l5_s1'
            ]
        elif condition == 'nfn':
            self.label = [
                'left_neural_foraminal_narrowing_l1_l2', f'left_neural_foraminal_narrowing_l2_l3', f'left_neural_foraminal_narrowing_l3_l4', f'left_neural_foraminal_narrowing_l4_l5', f'left_neural_foraminal_narrowing_l5_s1',
                'right_neural_foraminal_narrowing_l1_l2', f'right_neural_foraminal_narrowing_l2_l3', f'right_neural_foraminal_narrowing_l3_l4', f'right_neural_foraminal_narrowing_l4_l5', f'right_neural_foraminal_narrowing_l5_s1'
            ]
        self.id = df.loc[~(df[self.label].isna().any(axis=1)), 'study_id'].unique()
        self.id = list(set(self.id) - set([3637444890]))
        #self.id = [2773343225]
        #self.id = [1782095928]
        for l in self.label:
            df[l] = df[l].map({'Normal/Mild': 0, 'Moderate': 1, 'Severe': 2})

        self.wide_resize = v2.Resize((128, 224))
        #self.wide_resize = v2.Resize((224, 224))
        self.rec_resize = v2.Resize((256, 256))
        self.resize = v2.Resize((128, 128))
        self.resize_3d = v2.Resize((256, 256))
        self.pre_resize = v2.Resize((512, 512))
        self.wide_transforms = A.Compose([
            A.RandomBrightnessContrast(p=0.25),
            A.ShiftScaleRotate(shift_limit=0.1, scale_limit=(-0.1, 0.1), rotate_limit=20, border_mode=0, p=0.5),
            A.Resize(128, 224),
        ])
        self.rec_transforms = A.Compose([
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(p=0.25),
            A.ShiftScaleRotate(shift_limit=0.1, scale_limit=(-0.1, 0.1), rotate_limit=20, border_mode=0, p=0.5),
            A.Resize(256, 256),
        ])
    def __getitem__(self, index):
        study_id = self.id[index]
        res = {}
        if self.condition == 'scs':
            sagt2_img, ax_img, sagt1_img, sagt2_depth, ax_depth, sagt1_depth = self.for_scs(study_id)
            res['sagt2'] = sagt2_img.to(torch.float32)
            res['ax'] = ax_img.to(torch.float32)
            res['sagt1'] = sagt1_img.to(torch.float32)
            #res['sagt1'] = sagt1_img.to(torch.float32)
            res['sagt2_depth'] = sagt2_depth
            res['ax_depth'] = ax_depth
            res['sagt1_depth'] = sagt1_depth
            #res['sagt1'] = sagt1_img.to(torch.float32)

        label = self.df.loc[(self.df.study_id==study_id), self.label].values
        label = torch.tensor(label).squeeze().to(torch.long)
        res['label'] = label
        return res

    def crop(self, image, x, y, z, x_left, x_right, y_bottom, y_top, wide):
        size = [image[i].shape for i in z]
        data = torch.stack([torch.tensor(self.pre_resize(torch.tensor(image[i])[None, ...]).squeeze()[max(int((y/shape[0])*512-y_top), 0):int((y/shape[0])*512+y_bottom), max(int((x/shape[1])*512-x_left), 0): int((x/shape[1])*512+x_right)]) for i, shape in zip(z, size)])
        if wide:
            data = self.wide_resize(data)
        else:
            data = self.rec_resize(data)
        if self.usage == 'train':

            if wide:
                data = torch.tensor(self.wide_transforms(image=data.numpy().transpose((1, 2, 0)).astype(np.float32))['image'].transpose((2, 0, 1)))
            else:
                data = torch.tensor(self.rec_transforms(image=data.numpy().transpose((1, 2, 0)).astype(np.float32))['image'].transpose(2, 0, 1))
        return data

    def for_scs(self, study_id):
        sagt2_meta = self.meta.loc[(self.meta.study_id==study_id) & (self.meta.series_description=='Sagittal T2/STIR')]
        #display(sagt2_meta)
        ax_meta = self.meta.loc[(self.meta.study_id==study_id) & (self.meta.series_description=='Axial T2')]
        sagt1_meta = self.meta.loc[(self.meta.study_id==study_id) & (self.meta.series_description=='Sagittal T1')]
        #display(ax_meta)
        sagt2_meta = sagt2_meta.sort_values('ipp_x', ascending=True).reset_index(drop=True)
        ax_meta = ax_meta.sort_values('ipp_z', ascending=False).reset_index(drop=True)
        sagt1_meta = sagt1_meta.sort_values('ipp_x', ascending=True).reset_index(drop=True)
        sagt2_img = [self.normalize(self.load_dicom(f'/content/train_images/{row.study_id}/{row.series_id}/{row.instance_number}.dcm')) for _, row in sagt2_meta.iterrows()]
        ax_img = [self.normalize(self.load_dicom(f'/content/train_images/{row.study_id}/{row.series_id}/{row.instance_number}.dcm')) for _, row in ax_meta.iterrows()]
        sagt1_img = [self.normalize(self.load_dicom(f'/content/train_images/{row.study_id}/{row.series_id}/{row.instance_number}.dcm')) for _, row in sagt1_meta.iterrows()]
        sagt1_img = [img if (img.shape[0]> 1 and img.shape[1] > 1) else np.zeros((512, 512)) for img in sagt1_img]
        sagt2_sub_coor = self.coor.loc[(self.coor.study_id==study_id) & (self.coor.condition=='Spinal Canal Stenosis')]
        ax_right_sub_coor = self.coor.loc[(self.coor.study_id==study_id) & (self.coor.condition=='Right Subarticular Stenosis')]
        ax_left_sub_coor = self.coor.loc[(self.coor.study_id==study_id) & (self.coor.condition=='Left Subarticular Stenosis')]
        sagt1_sub_coor = self.coor.loc[(self.coor.study_id==study_id) & (self.coor.condition=='LR Neural Foraminal Narrowing')]

        # SAGITTAL T2
        sagt2_dict = {}
        sagt2_label_dict = {}
        for _, row in sagt2_sub_coor.iterrows():
            label = 2
            u = np.random.uniform(0, 1)
            if u < 0.7:
                z_shift = 0
            elif u > 0.7 and u < 0.95:
                z_shift = np.random.choice([-1, 1])
            else:
                z_shift = np.random.choice([-2, 2])
            if self.usage == 'train':
                y_shift = random.randint(-10, 10)
                x_shift = random.randint(-10, 10)
                #z_shift = random.randint(-1, 1)
            else:
                y_shift = 0
                x_shift = 0
                #z_shift = 0#random.randint(-2, 2)
            #display(sagt2_meta)
            #display(row)
            sagt2_label_dict[row.level] = label + z_shift
            #if label + z_shift < 0:
            #    sagt2_label_dict[row.level] = 0
            #elif label + z_shift > 2:
            #    sagt2_label_dict[row.level] = 2
            mid = sagt2_meta.loc[(sagt2_meta.series_id==row.series_id)&(sagt2_meta.instance_number==row.instance_number)].index[0]
            if row.level == 'L5/S1':
                ushift = 20
            else:
                ushift = 0
            z = [min(max(mid+w+z_shift, 0), len(sagt2_meta)-1) for w in range(-(self.sag_window-1)//2, ((self.sag_window-1)//2)+1)]
            sagt2_dict[row.level] = self.crop(sagt2_img, row.x+x_shift, row.y+y_shift, z, 96, 32, 40+ushift, 40-ushift, wide=True)
        sagt2_label = {l: torch.tensor(sagt2_label_dict.get(l, 2)).to(torch.long) for l in ['L1/L2', 'L2/L3', 'L3/L4', 'L4/L5', 'L5/S1']}
        # AXIAL T2
        #in_list = ax_meta.instance_number.tolist()
        ax_dict = {}
        ax_label_dict = {}
        if np.random.choice([0, 1]) == 0:
            ax_sub_coor = ax_right_sub_coor
            lrshift = +20
        else:
            ax_sub_coor = ax_left_sub_coor
            lrshift = -20
        for _, row in ax_sub_coor.iterrows():
            label = 2
            u = np.random.uniform(0, 1)
            if u < 0.5:
                z_shift = 0
            elif u > 0.5 and u < 0.85:
                z_shift = np.random.choice([-1, 1])
            else:
                z_shift = np.random.choice([-2, 2])
            if self.usage == 'train':
                y_shift = random.randint(-10, 10)
                x_shift = random.randint(-10, 10)
                #z_shift = random.randint(-1, 1)
            else:
                y_shift = 0
                x_shift = 0
                #z_shift = 0#random.randint(-1, 1)
            ax_label_dict[row.level] = label + z_shift
            ax_meta_sub = ax_meta.loc[(ax_meta.series_id==row.series_id)]
            ax_meta_sub_original_idx = ax_meta_sub.index.tolist()
            ax_meta_sub = ax_meta_sub.reset_index(drop=True)
            #display(ax_meta)
            #display(ax_meta_sub)
            #display(row)
            mid = ax_meta_sub.loc[(ax_meta_sub.instance_number==row.instance_number)].index[0]
            z = [min(max(mid+w+z_shift, 0), len(ax_meta_sub)-1) for w in range(-(self.ax_window-1)//2, ((self.ax_window-1)//2)+1)]
            ax_dict[row.level] = self.crop([ax_img[i] for i in range(len(ax_img)) if i in ax_meta_sub_original_idx], row.x+x_shift+lrshift, row.y+y_shift, z, 96, 96, 96, 96, wide=False)
        ax_label = {l: torch.tensor(ax_label_dict.get(l, 2)).to(torch.long) for l in ['L1/L2', 'L2/L3', 'L3/L4', 'L4/L5', 'L5/S1']}

        # SAGT1
        sagt1_dict = {}
        sagt1_label_dict = {}
        for _, row in sagt1_sub_coor.iterrows():
            label = 2
            u = np.random.uniform(0, 1)
            if u < 0.7:
                z_shift = 0
            elif u > 0.7 and u < 0.95:
                z_shift = np.random.choice([-1, 1])
            else:
                z_shift = np.random.choice([-2, 2])
            if self.usage == 'train':
                y_shift = random.randint(-10, 10)
                x_shift = random.randint(-10, 10)
                #z_shift = random.randint(-1, 1)
            else:
                y_shift = 0
                x_shift = 0
                #z_shift = 0#random.randint(-1, 1)
            if row.level == 'L5/S1':
                ushift = 20
            else:
                ushift = 0
            sagt1_label_dict[row.level] = label + z_shift
            sagt1_meta_sub = sagt1_meta.loc[(sagt1_meta.series_id==row.series_id)]
            sagt1_meta_sub_original_idx = sagt1_meta_sub.index.tolist()
            sagt1_meta_sub = sagt1_meta_sub.reset_index(drop=True)
            #display(sagt1_meta)
            #display(sagt1_meta_sub)
            #display(row)
            mid = sagt1_meta_sub.loc[(sagt1_meta_sub.instance_number==row.instance_number)].index[0]
            z = [min(max(mid+w+z_shift, 0), len(sagt1_meta_sub)-1) for w in range(-(self.sag_window-1)//2, ((self.sag_window-1)//2)+1)]
            #print(z)
            sagt1_dict[row.level] = self.crop([sagt1_img[i] for i in range(len(sagt1_img)) if i in sagt1_meta_sub_original_idx], row.x+x_shift, row.y+y_shift, z, 96, 64, 32+ushift, 32-ushift, wide=True)
        sagt1_label = {l: torch.tensor(sagt1_label_dict.get(l, 2)).to(torch.long) for l in ['L1/L2', 'L2/L3', 'L3/L4', 'L4/L5', 'L5/S1']}

        sagt2_img = [sagt2_dict.get(l, torch.zeros((self.sag_window, 128, 224))) for l in ['L1/L2', 'L2/L3', 'L3/L4', 'L4/L5', 'L5/S1']]
        ax_img = [ax_dict.get(l, torch.zeros((self.ax_window, 256, 256))) for l in ['L1/L2', 'L2/L3', 'L3/L4', 'L4/L5', 'L5/S1']]
        sagt1_img = [sagt1_dict.get(l, torch.zeros((self.sag_window, 128, 224))) for l in ['L1/L2', 'L2/L3', 'L3/L4', 'L4/L5', 'L5/S1']]
        return torch.stack(sagt2_img).contiguous(), torch.stack(ax_img).contiguous(), torch.stack(sagt1_img).contiguous(), sagt2_label, ax_label, sagt1_label

    def normalize(self, x):
        lower, upper = np.percentile(x, (1, 99))
        x = np.clip(x, lower, upper)
        x = x - np.min(x)
        x = x / np.max(x)
        return x

    def __len__(self):
        return len(self.id)

    def load_dicom(self, path):
        dicom = dcm.dcmread(path)
        data = dicom.pixel_array
        return data

    def check_position(self, first, last, pos):
        first_dcm = dcm.read_file(first)
        last_dcm = dcm.read_file(last)
        first_dcm = first_dcm.ImagePositionPatient[pos]
        last_dcm = last_dcm.ImagePositionPatient[pos]
        if pos == 0:
            return first_dcm > last_dcm
        elif pos == 2:
            return first_dcm < last_dcm
        else:
            raise ValueError


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class SCSDepthDetectLoss(nn.Module):
    def __init__(self):
        super(SCSDepthDetectLoss, self).__init__()

    def forward(self, outputs, targets):
        loss = 0
        for level in ['L1/L2', 'L2/L3', 'L3/L4', 'L4/L5', 'L5/S1']:
            #print(outputs[level].shape, targets[level].shape)
            _loss = nn.functional.cross_entropy(outputs[level], targets[level].reshape(-1))
            loss += _loss
        return loss/5

class NFNSSDepthDetectLoss(nn.Module):
    def __init__(self):
        super(NFNSSDepthDetectLoss, self).__init__()

    def forward(self, outputs, targets):
        loss = 0
        for level in ['left_L1/L2', 'left_L2/L3', 'left_L3/L4', 'left_L4/L5', 'left_L5/S1',
                  'right_L1/L2', 'right_L2/L3', 'right_L3/L4', 'right_L4/L5', 'right_L5/S1']:
            _loss = nn.functional.cross_entropy(outputs[level], targets[level].reshape(-1))
            loss += _loss
        return loss/10
class SCSNFNSSLoss(nn.Module):
    def __init__(self, is_train=False):
        super(SCSNFNSSLoss, self).__init__()
        self.loss = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 4.0]).to(device))
        self.aux_loss_ax = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 4.0]).to(device))
        self.aux_loss_sagt2 = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 4.0]).to(device))
        self.aux_loss_sagt1 = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 4.0]).to(device))
        self.is_train = is_train
    def forward(self, outputs, targets, ax=None, sagt2=None, sagt1=None):
        targets = targets.reshape(-1)
        #print(outputs, targets, outputs.shape, targets.shape)
        ax_loss = 0
        sagt2_loss = 0
        sagt1_loss = 0
        num_loss = 1
        loss = self.loss(outputs, targets)
        if ax is not None:
            ax_loss = self.aux_loss_ax(ax, targets)
            if self.is_train:
                loss += ax_loss*0.5
                num_loss += 0.5
        if sagt2 is not None:
            sagt2_loss = self.aux_loss_sagt2(sagt2, targets)
            if self.is_train:
                loss += sagt2_loss*0.5
                num_loss += 0.5
        if sagt1 is not None:
            sagt1_loss = self.aux_loss_sagt1(sagt1, targets)
            if self.is_train:
                loss += sagt1_loss*0.5
                num_loss += 0.5
        #print(loss)
        loss = loss/num_loss
        return loss, ax_loss, sagt2_loss, sagt1_loss

In [ ]:
class LSTMMIL(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(LSTMMIL, self).__init__()
        self.lstm = nn.LSTM(input_dim, input_dim//2, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
        self.aux_attention = nn.Sequential(
            nn.Tanh(),
            nn.Linear(input_dim, 1)
        )
        self.attention = nn.Sequential(
            nn.Tanh(),
            nn.Linear(input_dim, 1)
        )
    def forward(self, bags):
        """
        Args:
            bags: (batch_size, num_instances, input_dim)

        Returns:
            logits: (batch_size, num_classes)
        """
        batch_size, num_instances, input_dim = bags.size()
        bags_lstm, _ = self.lstm(bags)
        attn_scores = self.attention(bags_lstm).squeeze(-1)
        aux_attn_scores = self.aux_attention(bags_lstm).squeeze(-1)
        attn_weights = torch.softmax(attn_scores, dim=-1)
        weighted_instances = torch.bmm(attn_weights.unsqueeze(1), bags_lstm).squeeze(1)  # (batch_size, input_dim)
        return weighted_instances, aux_attn_scores


class SCSMIL(nn.Module):
    def __init__(self):
        super().__init__()
        self.sagt2_encoder = timm.create_model('convnext_small.fb_in22k_ft_in1k_384', in_chans=1, pretrained=True, num_classes=0)
        #self.sagt1_encoder = timm.create_model('convnext_base.fb_in22k_ft_in1k_384', in_chans=1, pretrained=True, num_classes=0)
        self.ax_encoder = timm.create_model('convnext_small.fb_in22k_ft_in1k_384', in_chans=1, pretrained=True, num_classes=0)
        self.sagt2_flatten = nn.Sequential(nn.AdaptiveAvgPool2d((1,1)),
                                    nn.Flatten(1))
        self.ax_flatten = nn.Sequential(nn.AdaptiveAvgPool2d((1,1)),
                                    nn.Flatten(1))
        self.sagt2_num_features = self.sagt2_encoder.num_features
        self.ax_num_features = self.ax_encoder.num_features
        self.sagt2_head = LSTMMIL(self.sagt2_num_features, 512, 3)
        self.ax_head = LSTMMIL(self.ax_num_features, 512, 3)
        self.out = nn.Linear(self.sagt2_num_features + self.ax_num_features, 3)
        self.aux_out = nn.Linear(self.sagt2_num_features, 3)
        #self.dropout = nn.Dropout(0.1)
    def forward(self, ax, sagt2, sagt1):
        ax_shape = ax.shape
        ax = ax.reshape(ax_shape[0]*ax_shape[1]*ax_shape[2], 1, ax_shape[-2], ax_shape[-1])
        ax = self.ax_encoder.forward_features(ax)
        ax = self.ax_flatten(ax)
        ax = ax.reshape(ax_shape[0]*ax_shape[1], ax_shape[2], -1)
        ax_weighted_sum, ax_attn = self.ax_head(ax)
        ax_attn = ax_attn.reshape(ax_shape[0], ax_shape[1], -1)

        sagt2_shape = sagt2.shape
        sagt2 = sagt2.reshape(sagt2_shape[0]*sagt2_shape[1]*sagt2_shape[2], 1, sagt2_shape[-2], sagt2_shape[-1])
        sagt2 = self.sagt2_encoder.forward_features(sagt2)
        sagt2 = self.sagt2_flatten(sagt2)
        sagt2 = sagt2.reshape(sagt2_shape[0]*sagt2_shape[1], sagt2_shape[2], -1)
        sagt2_weighted_sum, sagt2_attn = self.sagt2_head(sagt2)
        sagt2_attn = sagt2_attn.reshape(sagt2_shape[0], sagt2_shape[1], -1)
        out = torch.cat([ax_weighted_sum, sagt2_weighted_sum], dim=1)
        out = self.out(out)
        sagt2_out = self.aux_out(sagt2_weighted_sum)
        ax_out = self.aux_out(ax_weighted_sum)
        ax_attn = {'L1/L2': ax_attn[:, 0, :], 'L2/L3': ax_attn[:, 1, :], 'L3/L4': ax_attn[:, 2, :], 'L4/L5': ax_attn[:, 3, :], 'L5/S1': ax_attn[:, 4, :]}
        sagt2_attn = {'L1/L2': sagt2_attn[:, 0, :], 'L2/L3': sagt2_attn[:, 1, :], 'L3/L4': sagt2_attn[:, 2, :], 'L4/L5': sagt2_attn[:, 3, :], 'L5/S1': sagt2_attn[:, 4, :]}
        return out, ax_attn, sagt2_attn, sagt2_out, ax_out


In [ ]:
from traitlets.config.loader import ConfigError
VAL_SCORE = []

class ClassModule(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.condition = config['task']['condition']
        if config['task']['condition'] == 'scs':
            self.loss_module = SCSNFNSSLoss(is_train=True)
            self.val_loss_module = SCSNFNSSLoss(is_train=False)
            self.scs_depth_loss_module = SCSDepthDetectLoss()
            self.nfn_ss_depth_loss_module = SCSDepthDetectLoss()
            self.sagt1_depth_loss_module = SCSDepthDetectLoss()
            self.model = SCSMIL()

        self.val_step_outputs = []
        self.val_step_ax_outputs = []
        self.val_step_sagt1_outputs = []
        self.val_step_sagt2_outputs = []
        self.val_step_labels = []
        self.aux_log = {}
        #self.ema = ExponentialMovingAverage(self.model.parameters(), decay=0.995)
        #self.ema.to(device)

        #self.model = torch.optim.swa_utils.AveragedModel(self.model,
        #                                                 multi_avg_fn=torch.optim.swa_utils.get_ema_multi_avg_fn(0.999))

    def forward(self, batch):
        preds = self.model(batch)
        return preds

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), **self.config['model']["optimizer_params"])
        if self.config['model']["scheduler"]["name"] == "CosineAnnealingLR":
            scheduler = CosineAnnealingLR(
                optimizer,
                **self.config['model']["scheduler"]["params"]["CosineAnnealingLR"],
            )
            lr_scheduler_dict = {"scheduler": scheduler, "interval": "step"}
            return {"optimizer": optimizer, "lr_scheduler": lr_scheduler_dict}
        elif self.config['model']["scheduler"]["name"] == "ReduceLROnPlateau":
            scheduler = ReduceLROnPlateau(
                optimizer,
                **self.config['model']["scheduler"]["params"]["ReduceLROnPlateau"],
            )
            lr_scheduler = {"scheduler": scheduler, "monitor": "val_loss"}
            return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}
        elif self.config['model']['scheduler']['name'] == 'ChrisLR':
            scheduler = ChrisLR(
                optimizer,
                **self.config['model']["scheduler"]["params"]["ChrisLR"],
            )
            lr_scheduler = {"scheduler": scheduler, "monitor": "val_loss"}
            return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}
        elif self.config['model']['scheduler']['name'] == 'cosine_with_warmup':
            print('cosine with warmup')
            print(self.config['model']['scheduler']['params']['cosine_with_warmup'])
            scheduler = transformers.get_cosine_schedule_with_warmup(
                optimizer,
                **self.config['model']['scheduler']['params']['cosine_with_warmup'],
            )
            lr_scheduler_dict = {"scheduler": scheduler, "interval": "step"}
            return {"optimizer": optimizer, "lr_scheduler": lr_scheduler_dict}
        else:
            return {"optimizer": optimizer}

    def training_step(self, batch, batch_idx):
        if self.condition == 'scs':
            ax, sagt2, sagt1, label, ax_depth, sagt2_depth, sagt1_depth = batch['ax'], batch['sagt2'], batch['sagt1'], batch['label'], batch['ax_depth'], batch['sagt2_depth'], batch['sagt1_depth']
            preds, ax_depth_pred, sagt2_depth_pred, sagt2_preds, ax_preds = self.model(ax, sagt2, sagt1)
            loss, _, _, _ = self.loss_module(preds, label, ax=ax_preds, sagt2=sagt2_preds)
            self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, batch_size=config['train_bs'])
            ax_depth_loss = self.nfn_ss_depth_loss_module(ax_depth_pred, ax_depth)
            sagt2_depth_loss = self.scs_depth_loss_module(sagt2_depth_pred, sagt2_depth)
            self.log('train ax depth loss', ax_depth_loss, on_step=False, on_epoch=True, prog_bar=True)
            self.log('train sagt2 depth loss', sagt2_depth_loss, on_step=False, on_epoch=True, prog_bar=True)
            loss = loss + ax_depth_loss*(1/2) + sagt2_depth_loss*(1/2)

        for param_group in self.trainer.optimizers[0].param_groups:
            lr = param_group["lr"]
        self.log("lr", lr, on_step=True, on_epoch=False, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        """Add TTA"""
        if self.condition == 'scs':
            ax, sagt2, sagt1, label, ax_depth, sagt2_depth, sagt1_depth = batch['ax'], batch['sagt2'], batch['sagt1'], batch['label'], batch['ax_depth'], batch['sagt2_depth'], batch['sagt1_depth']
            preds, ax_depth_pred, sagt2_depth_pred, _, _ = self.model(ax=ax, sagt2=sagt2, sagt1=sagt1)
            loss, _, _, _ = self.val_loss_module(preds, label)
            self.log("val_loss_mean", loss, on_step=False, on_epoch=True, prog_bar=True)
            ax_depth_loss = self.nfn_ss_depth_loss_module(ax_depth_pred, ax_depth)
            sagt2_depth_loss = self.scs_depth_loss_module(sagt2_depth_pred, sagt2_depth)
            self.log('ax depth loss', ax_depth_loss, on_step=False, on_epoch=True, prog_bar=True)
            self.log('sagt2 depth loss', sagt2_depth_loss, on_step=False, on_epoch=True, prog_bar=True)
            self.val_step_outputs.append(preds)
            self.val_step_labels.append(label)

    def on_validation_epoch_end(self):

        all_preds = torch.cat(self.val_step_outputs).float()
        all_labels = torch.cat(self.val_step_labels)
        all_labels = all_labels.to(torch.long)
        val_loss, ax, sagt2, sagt1 = self.val_loss_module(all_preds, all_labels)#, ax=all_preds_ax#, sagt1=all_preds_sagt1)
        self.log("val_loss", val_loss, on_step=False, on_epoch=True, prog_bar=True)
        print(val_loss)
        self.val_step_outputs.clear()
        self.val_step_labels.clear()
        self.val_step_ax_outputs.clear()
        self.val_step_sagt1_outputs.clear()
        self.val_step_sagt2_outputs.clear()

        #print(self.aux_log['ax_loss'], self.logger.aux_log['sagt1_loss'], self.logger.log_dir['val_loss'])
        if self.trainer.global_rank == 0:
            print(f"\nEpoch: {self.current_epoch}", flush=True)
        return

    def optimizer_step(self, *args, **kwargs):
        super().optimizer_step(*args, **kwargs)
        #self.ema.update()

In [ ]:
train = pd.read_csv('/content/train.csv')
train_dummy = pd.read_csv('/content/train.csv')
train_series = pd.read_csv('/content/train_series_descriptions.csv')
train_coor = pd.read_csv('/content/expanded_train_label_coordinates.csv')
train_coor['y'] = pd.to_numeric(train_coor['y'], errors='coerce')
train_coor['x'] = pd.to_numeric(train_coor['x'], errors='coerce')
train_coor = train_coor.loc[(~train_coor.x.isna()) & (~train_coor.y.isna())]
train_meta = pd.read_parquet('/content/meta.parquet')

In [ ]:
train_dummy = train_dummy.fillna('Normal/Mild')

In [ ]:
label = train.columns[1:]
kfold = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for i, (train_idx, valid_idx) in enumerate(kfold.split(train_dummy, train_dummy[label])):
    train_dummy.loc[valid_idx, 'fold'] = i
train_series = train_series.merge(train_dummy[['study_id', 'fold']], on='study_id')
train_coor = train_coor.merge(train_dummy[['study_id', 'fold']], on='study_id')
train = train.merge(train_dummy[['study_id', 'fold']], on='study_id')
train_meta = train_meta.merge(train_dummy[['study_id', 'fold']], on='study_id')

In [ ]:
T_MAX = config["model"]["scheduler"]["params"]["CosineAnnealingLR"]["T_max"]
num_training_steps = config["model"]["scheduler"]["params"]["cosine_with_warmup"]["num_training_steps"]
num_warmup_steps = config["model"]["scheduler"]["params"]["cosine_with_warmup"]["num_warmup_steps"]

In [ ]:
%%time
import warnings
warnings.filterwarnings("ignore")
#######################CLASSIFY#############################
print(config['task']['condition'])
print(T_MAX)
#for i in [5]:
for i in [0, 1, 2, 3, 4]:
    train_df = train.loc[train.fold != i]
    valid_df = train.loc[train.fold == i]
    train_df = train_df.drop('fold', axis=1)
    valid_df = valid_df.drop('fold', axis=1)
    train_series_df = train_series.loc[train_series.fold != i]
    valid_series_df = train_series.loc[train_series.fold == i]
    train_series_df = train_series_df.drop('fold', axis=1)
    valid_series_df = valid_series_df.drop('fold', axis=1)
    if config['task']['condition'] != 'nfn':
        train_coor_df = train_coor.loc[train_coor.fold != i]
        valid_coor_df = train_coor.loc[train_coor.fold == i]
        train_coor_df = train_coor_df.drop('fold', axis=1)
        valid_coor_df = valid_coor_df.drop('fold', axis=1)
    else:
        train_coor_df = train_coor.loc[train_coor.fold != i]
        valid_coor_df = train_coor.loc[train_coor.fold == i]
        #train_coor_df = coor_for_nfn.loc[coor_for_nfn.fold != i]
        #valid_coor_df = coor_for_nfn.loc[coor_for_nfn.fold == i]
        train_coor_df = train_coor_df.drop('fold', axis=1)
        valid_coor_df = valid_coor_df.drop('fold', axis=1)
    train_meta_df = train_meta.loc[train_meta.fold != i]
    valid_meta_df = train_meta.loc[train_meta.fold == i]
    train_meta_df = train_meta_df.drop('fold', axis=1)
    valid_meta_df = valid_meta_df.drop('fold', axis=1)

    dataset_train = ClassDataset(train_df, train_series_df, train_coor_df, train_meta_df, condition=config['task']['condition'], usage='train')
    dataset_validation = ClassDataset(valid_df, valid_series_df, valid_coor_df, valid_meta_df, condition=config['task']['condition'], usage='valid')
    #dataset_train = dataset_train[:100]
    #dataset_validation = dataset_validation[:100]

    data_loader_train = DataLoader(
        dataset_train,
        batch_size=config["train_bs"],
        shuffle=True,
        num_workers=4,
        pin_memory=True,
    )
    data_loader_validation = DataLoader(
        dataset_validation,
        batch_size=config["valid_bs"],
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )
    print(len(dataset_train), len(dataset_validation))
    if config['task']['condition'] == 'scs':
        checkpoint_callback = ModelCheckpoint(
            save_weights_only=True,
            monitor="val_loss",
            dirpath=config["output_dir"],
            mode='min',
            filename=f"_{config['task']['condition']}_{config['task']['kind']}_5ch_axsagt2-lstm-mil_auxloss_auxdepth_convnext-s-{i}",
            save_top_k=config["save_topk"],
            verbose=1,
        )

    progress_bar_callback = TQDMProgressBar(
        refresh_rate=config["progress_bar_refresh_rate"]
    )

    early_stop_callback = EarlyStopping(**config["early_stop"])
    con = config['task']['condition']
    kind = config['task']['kind']
    wandb_logger = WandbLogger(project=f'rsna_spine_{con}_{kind}', # group runs in "MNIST" project
                            log_model=False,  # log all new checkpoints during training
                            name=f'convnext-s_lstmmil-fold{i}')
    trainer = pl.Trainer(
        val_check_interval=0.5,
        logger=wandb_logger,
        callbacks=[checkpoint_callback, early_stop_callback, progress_bar_callback],
        **config["trainer"],
    )

    config["model"]["scheduler"]["params"]["CosineAnnealingLR"]["T_max"] = T_MAX*len(data_loader_train)/config["trainer"]["devices"]
    config["model"]["scheduler"]["params"]["cosine_with_warmup"]["num_training_steps"] = int(num_training_steps*len(data_loader_train)/config["trainer"]["devices"])
    config["model"]["scheduler"]["params"]["cosine_with_warmup"]["num_warmup_steps"] = int(num_warmup_steps*len(data_loader_train)/config["trainer"]["devices"])
    model = ClassModule(config=config)
    trainer.fit(model, data_loader_train, data_loader_validation)
    wandb.finish()

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


scs
20
1578 395


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: syujinko. Use `wandb login --relogin` to force relogin


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                     | Type               | Params | Mode 
------------------------------------------------------------------------
0 | loss_module              | SCSNFNSSLoss       | 0      | train
1 | val_loss_module          | SCSNFNSSLoss       | 0      | train
2 | scs_depth_loss_module    | SCSDepthDetectLoss | 0      | train
3 | nfn_ss_depth_loss_module | SCSDepthDetectLoss | 0      | train
4 | sagt1_depth_loss_module  | SCSDepthDetectLoss | 0      | train
5 | model                    | SCSMIL             | 113 M  | train
------------------------------------------------------------------------
113 M     Trainable params
0         Non-trainable params
113 M     Total params
452.374   Total estimated model params size (MB)
968       Modules in train mode
0         Modules in eval mode


cosine with warmup
{'num_training_steps': 6312, 'num_warmup_steps': 789, 'num_cycles': 0.5, 'last_epoch': -1}


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

tensor(1.0712, device='cuda:0')

Epoch: 0


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined